## Training basic linear models with integrated feature + segmentation for census prediction 

### 1. PCA prediction on quantile ranges of sociodemographics

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# -------------------------
# 1️⃣ Load PCA-Transformed Features
# -------------------------
folder_path = "/Users/etoileboots/Desktop/CAPSTONE/res_net/features_and_visualizations"  # Update if necessary
feature_files = [f for f in os.listdir(folder_path) if f.endswith(".npy")]

tract_features = {}

for file in feature_files:
    tract_name = file.replace(".npy", "")  # Extract tract identifier
    file_path = os.path.join(folder_path, file)
    features_dict = np.load(file_path, allow_pickle=True).item()
    tract_features[tract_name] = np.array(list(features_dict.values()))  # Convert to NumPy array

# Combine all feature vectors for PCA
all_features = np.vstack(list(tract_features.values()))

# Apply PCA - Keep enough components to explain 95% variance
pca = PCA(n_components=0.95)
pca.fit(all_features)

# -------------------------
# 2️⃣ Transform and Store Tract-Level PCA Features
# -------------------------
tract_pca_results = {}

for tract in tract_features:
    transformed_features = pca.transform(tract_features[tract])
    tract_pca_results[tract] = transformed_features.mean(axis=0)  # Use mean PCA features per tract

# Convert to DataFrame
pca_df = pd.DataFrame.from_dict(tract_pca_results, orient="index")

# Ensure correct column naming
pca_df.columns = [f"PCA_{i+1}" for i in range(pca_df.shape[1])]

# Convert index to a proper column for merging
pca_df.index.name = "GEOID20"
pca_df.reset_index(inplace=True)

# Print Results
print("PCA-transformed feature DataFrame shape:", pca_df.shape)
print(pca_df.head())

In [ ]:
# rename the geiod data frame 
pca_df.index = pca_df.index.astype(str)  # Ensure index is string type
pca_df["GEOID20"] = pca_df["GEOID20"].astype(str).str.replace("features_", "", regex=False)
pca_df.head()

In [ ]:
# SOCIODEMOGRAPHICS
# -------------------------
# 2️⃣ Load Sociodemographic Data
# -------------------------
csv_path = "/Volumes/MRDALLMAYR/data/census_data/census_data_harris_county_2022.csv"  # Update if necessary
socio_df = pd.read_csv(csv_path, dtype={"GEOID20": str})  # Load as string to avoid type mismatches

# Select relevant columns (remove categorical or ID columns)
target_columns = [
    "median_income", "pct_income_below_poverty", "pct_income_above_poverty", 
    "pct_no_vehicle", "pct_public_transport", "pct_commute_more_than_60"
]

socio_df = socio_df[["GEOID20"] + target_columns]  # Keep only relevant data
socio_df.head()

In [ ]:

# -------------------------
# 3️⃣ Merge PCA Features and Sociodemographic Data
# -------------------------

socio_df["GEOID20"] = socio_df["GEOID20"].astype(str)
pca_df["GEOID20"] = pca_df["GEOID20"].astype(str)
socio_PCA_df = socio_df.merge(pca_df, on="GEOID20")
socio_PCA_df.head()

In [ ]:
# Compute correlation matrix for all numeric columns
corr_matrix = socio_PCA_df.select_dtypes(include=[np.number]).corr()

# Extract correlations between PCA features and sociodemographic variables
pca_feature_cols = [col for col in socio_PCA_df.columns if col.startswith("PCA")]
socio_cols = ["median_income", "pct_income_below_poverty", "pct_income_above_poverty",
              "pct_no_vehicle", "pct_public_transport", "pct_commute_more_than_60"]  # Adjust based on dataset

# Subset correlation matrix to show only PCA vs. Sociodemographic correlations
correlation_results = corr_matrix.loc[pca_feature_cols, socio_cols]

# Sort PCA components by their highest absolute correlation with any target variable
correlation_rank = correlation_results.abs().max(axis=1).sort_values(ascending=False)

# Print the top 10 PCA components with the highest correlation to any sociodemographic variable
print("\nTop PCA Features with Highest Correlation to Sociodemographic Factors:")
print(correlation_rank.head(20))


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Compute absolute correlations and rank PCA features
correlation_rank = correlation_results.abs().max(axis=1).sort_values(ascending=False)

# Select the top 10 most correlated PCA features
top_pca_features = correlation_rank.head(10).index

# Subset correlation matrix for only the top PCA components
top_correlation_results = correlation_results.loc[top_pca_features]

# Plot heatmap for only the top PCA features
plt.figure(figsize=(12, 6))
sns.heatmap(top_correlation_results, annot=True, fmt=".2f", cmap="coolwarm", center=0)
plt.title("Top 10 PCA Features Correlated with Sociodemographic Variables")
plt.show()


## Model training

In [ ]:
# turning median income into a binary classification problem 
# Check if there are any negative median income values
negative_values = socio_PCA_df[socio_PCA_df["median_income"] < 0]

if not negative_values.empty:
    print("Warning: Found negative median income values!")
    print(negative_values)
else:
    print("No negative median income values found.")

# Remove rows where median_income is negative
socio_PCA_df = socio_PCA_df[socio_PCA_df["median_income"] >= 0]

# Reset index after removal
socio_PCA_df.reset_index(drop=True, inplace=True)

# Verify that no negative values remain
print("Negative median income values remaining:", (socio_PCA_df["median_income"] < 0).sum())


In [ ]:
income_median = socio_PCA_df["median_income"].median()
print(f"Median Income Threshold: {income_median} USD, annually")


In [ ]:
# create a binary index variable: 
socio_PCA_df["median_income_binary"] = (socio_PCA_df["median_income"] >= income_median).astype(int)
print(socio_PCA_df["median_income_binary"].value_counts())


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

# -------------------------
# 1️⃣ Prepare Data
# -------------------------
# Select PCA features
pca_feature_cols = [col for col in socio_PCA_df.columns if col.startswith("PCA")]
X = socio_PCA_df[pca_feature_cols].values  # Convert to NumPy array
y = socio_PCA_df["median_income_binary"].values  # Binary labels (0 or 1)

# Standardize features for SVM & Logistic Regression
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

print(f"Training set shape: {X_train.shape}, Testing set shape: {X_test.shape}")

# -------------------------
# 2️⃣ Train and Evaluate Models
# -------------------------
models = {
    "Logistic Regression": LogisticRegression(),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Support Vector Machine": SVC(kernel="rbf", probability=True),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric="logloss")
}

results = {}

for name, model in models.items():
    print(f"\nTraining {name}...")
    model.fit(X_train, y_train)  # Train model
    
    y_pred = model.predict(X_test)  # Make predictions
    
    # Compute accuracy & classification report
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True)

    # Store results
    results[name] = {
        "Accuracy": accuracy,
        "Precision": report["1"]["precision"],  # Precision for label 1 (higher income)
        "Recall": report["1"]["recall"],        # Recall for label 1
        "F1-Score": report["1"]["f1-score"]      # F1-score for label 1
    }

# -------------------------
# 3️⃣ Display Model Performance
# -------------------------
results_df = pd.DataFrame(results).T  # Convert to DataFrame
print("\nModel Performance Summary:")
print(results_df)

# -------------------------
# 4️⃣ Visualize Model Performance
# -------------------------
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
results_df.plot(kind="bar", figsize=(12,6), colormap="coolwarm")
plt.title("Model Performance Comparison on Binary Median Income Binary")
plt.ylabel("Score")
plt.xticks(rotation=45)
plt.grid(axis="y")
plt.legend(loc="lower right")
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVC
from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import accuracy_score, classification_report, mean_absolute_error, r2_score

# -------------------------
# 1️⃣ Define Target Variables & Prepare Data
# -------------------------
# Select PCA features
pca_feature_cols = [col for col in socio_PCA_df.columns if col.startswith("PCA")]
X = socio_PCA_df[pca_feature_cols].values  # Convert PCA features to NumPy array

# Define **classification** (binary) and **regression** (continuous) target variables
binary_targets = ["median_income"]  # Targets that need binary classification
regression_targets = [
    "pct_income_below_poverty", "pct_income_above_poverty",
    "pct_no_vehicle", "pct_public_transport", "pct_commute_more_than_60"
]  # Continuous regression targets

# Convert each binary target variable into 0 = below median, 1 = above median
for target in binary_targets:
    median_value = socio_PCA_df[target].median()
    socio_PCA_df[f"{target}_binary"] = (socio_PCA_df[target] >= median_value).astype(int)

# Standardize features for better model performance
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# -------------------------
# 2️⃣ Train and Evaluate Classification Models (For Binary Targets)
# -------------------------
classification_models = {
    "Logistic Regression": LogisticRegression(),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Support Vector Machine": SVC(kernel="rbf", probability=True),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric="logloss")
}

classification_results = {}

for target in binary_targets:
    print(f"\n=== Training Classification Models for {target}_binary ===")

    # Define binary target variable
    y = socio_PCA_df[f"{target}_binary"].values

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

    results = {}

    for name, model in classification_models.items():
        print(f"\nTraining {name} on {target}...")
        model.fit(X_train, y_train)  # Train model

        y_pred = model.predict(X_test)  # Make predictions

        # Compute accuracy & classification report
        accuracy = accuracy_score(y_test, y_pred)
        report = classification_report(y_test, y_pred, output_dict=True)

        # Store results
        results[name] = {
            "Accuracy": accuracy,
            "Precision": report["1"]["precision"],  # Precision for label 1
            "Recall": report["1"]["recall"],        # Recall for label 1
            "F1-Score": report["1"]["f1-score"]      # F1-score for label 1
        }

    # Store classification results
    classification_results[target] = results

# -------------------------
# 3️⃣ Train and Evaluate Regression Models (For Continuous Targets)
# -------------------------
regression_models = {
    "Linear Regression": LinearRegression(),
    "Random Forest Regressor": RandomForestRegressor(n_estimators=100, random_state=42),
    "XGBoost Regressor": XGBRegressor(objective="reg:squarederror", random_state=42)
}

regression_results = {}

for target in regression_targets:
    print(f"\n=== Training Regression Models for {target} ===")

    # Define continuous target variable
    y = socio_PCA_df[target].values

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    results = {}

    for name, model in regression_models.items():
        print(f"\nTraining {name} on {target}...")
        model.fit(X_train, y_train)  # Train model

        y_pred = model.predict(X_test)  # Make predictions

        # Compute MAE and R^2 Score
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        # Store results
        results[name] = {
            "Mean Absolute Error": mae,
            "R² Score": r2
        }

    # Store regression results
    regression_results[target] = results

# -------------------------
# 4️⃣ Display and Visualize Model Performance
# -------------------------
# Classification results
for target, results in classification_results.items():
    results_df = pd.DataFrame(results).T
    print(f"\nClassification Model Performance for {target}_binary:")
    print(results_df)

    # Visualization
    plt.figure(figsize=(10, 5))
    results_df.plot(kind="bar", figsize=(12,6), colormap="coolwarm")
    plt.title(f"Classification Model Performance for {target}_binary")
    plt.ylabel("Score")
    plt.xticks(rotation=45)
    plt.grid(axis="y")
    plt.legend(loc="lower right")
    plt.show()

# Regression results
for target, results in regression_results.items():
    results_df = pd.DataFrame(results).T
    print(f"\nRegression Model Performance for {target}:")
    print(results_df)

    # Visualization
    plt.figure(figsize=(10, 5))
    results_df.plot(kind="bar", figsize=(12,6), colormap="coolwarm")
    plt.title(f"Regression Model Performance for {target}")
    plt.ylabel("Score")
    plt.xticks(rotation=45)
    plt.grid(axis="y")
    plt.legend(loc="upper right")
    plt.show()


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

# -------------------------
# 1️⃣ Prepare Data
# -------------------------
# Select PCA features as input
pca_feature_cols = [col for col in socio_PCA_df.columns if col.startswith("PCA")]
X = socio_PCA_df[pca_feature_cols].values  # Convert DataFrame to NumPy array

# Target variable
y = socio_PCA_df["median_income_binary"].values  # Binary labels (0 or 1)

# Standardize the PCA features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)  # Normalize features for better performance

# Split into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

print(f"Training set: {X_train.shape}, Testing set: {X_test.shape}")

# -------------------------
# 2️⃣ Build the Neural Network
# -------------------------
model = keras.Sequential([
    keras.layers.Dense(256, activation="relu", input_shape=(X_train.shape[1],)),  # First hidden layer
    keras.layers.Dropout(0.3),  # Dropout to prevent overfitting
    keras.layers.Dense(128, activation="relu"),  # Second hidden layer
    keras.layers.Dropout(0.2),  # Another Dropout layer
    keras.layers.Dense(64, activation="relu"),  # Third hidden layer
    keras.layers.Dense(1, activation="sigmoid")  # Output layer (binary classification)
])

# Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# -------------------------
# 3️⃣ Train the Model
# -------------------------
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test))

# -------------------------
# 4️⃣ Evaluate the Model
# -------------------------
# Make predictions
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)  # Convert probabilities to binary labels

# Compute accuracy and classification report
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.4f}")
print("Classification Report:\n", classification_report(y_test, y_pred))

# -------------------------
# 5️⃣ Plot Training Progress
# -------------------------
import matplotlib.pyplot as plt

# Plot training & validation accuracy
plt.figure(figsize=(10,5))
plt.plot(history.history['accuracy'], label="Train Accuracy")
plt.plot(history.history['val_accuracy'], label="Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Model Accuracy Over Epochs")
plt.legend()
plt.show()

# Plot training & validation loss
plt.figure(figsize=(10,5))
plt.plot(history.history['loss'], label="Train Loss")
plt.plot(history.history['val_loss'], label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Model Loss Over Epochs")
plt.legend()
plt.show()


In [ ]:

# Separate features (X) and target variables (y)
X = df.drop(columns=["GEOID20"] + target_columns)  # Keep only PCA features
y = df[target_columns]  # Targets for regression

# -------------------------
# 4️⃣ Train-Test Split
# -------------------------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# -------------------------
# 5️⃣ Train Regression Model
# -------------------------
from sklearn.preprocessing import StandardScaler

# Standardize features (X)
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

# Standardize targets (y)
scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train)  # Fit & transform training targets
y_test_scaled = scaler_y.transform(y_test)  # Transform test targets

# Train model on scaled data
lr = LinearRegression()
lr.fit(X_train_scaled, y_train_scaled)

# Predict and inverse transform
y_pred_scaled = lr.predict(X_test_scaled)
y_pred = scaler_y.inverse_transform(y_pred_scaled)  # Convert back to original scale

# Compute separate MSE for each target
mse_values = {}
for i, target in enumerate(y.columns):
    mse = mean_squared_error(y_test[target], y_pred[:, i])
    mse_values[target] = mse
    print(f"MSE for {target}: {mse:.4f}")

# -------------------------
# 6️⃣ Train a Random Forest Model (for comparison)
# -------------------------
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train_scaled, y_train)
y_pred_rf = rf.predict(X_test_scaled)

mae_rf = mean_absolute_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print(f"Random Forest - MAE: {mae_rf:.4f}, R² Score: {r2_rf:.4f}")

# -------------------------
# 7️⃣ Plot Feature Importance from PCA Components
# -------------------------
plt.figure(figsize=(12, 6))
plt.bar(range(1, X_train.shape[1] + 1), np.abs(lr.coef_).mean(axis=0))  # Average importance across targets
plt.xlabel("PCA Component Number")
plt.ylabel("Feature Importance (Absolute Coefficients)")
plt.title("Feature Importance of PCA Components in Regression Model")
plt.grid()
plt.show()


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

# Define MLP Model
model = keras.Sequential([
    layers.Dense(128, activation="relu", input_shape=(X_train.shape[1],)),
    layers.Dense(64, activation="relu"),
    layers.Dense(y_train.shape[1])  # Output layer (same as number of target variables)
])

model.compile(optimizer="adam", loss="mse")

# Train the model
model.fit(X_train_scaled, y_train_scaled, epochs=50, batch_size=16, validation_split=0.2)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plot residuals for each target variable
for i, target in enumerate(y.columns):
    plt.figure(figsize=(8, 5))
    sns.histplot(y_test[target] - y_pred[:, i], bins=30, kde=True)
    plt.xlabel("Residuals")
    plt.ylabel("Frequency")
    plt.title(f"Residual Distribution for {target}")
    plt.axvline(0, color='r', linestyle='dashed')  # Ideal residuals should be centered at 0
    plt.show()


In [ ]:
# Plot training vs. validation loss over epochs
plt.figure(figsize=(10, 5))
plt.plot(model.history.history['loss'], label='Training Loss')
plt.plot(model.history.history['val_loss'], label='Validation Loss')
plt.xlabel("Epochs")
plt.ylabel("Loss (MSE)")
plt.title("Training vs Validation Loss")
plt.legend()
plt.show()
